In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
baseline = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\baseline_data.xlsx")

In [4]:
baseline['race_datetime'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: race_datetime
Non-Null Count  Dtype         
--------------  -----         
100 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 932.0 bytes


In [7]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,Team,Winner Time,Avg Speed (km/h),2nd Place,3rd Place,...,Won How,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race coords
0,94,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,Quick-Step Floors,6:31:40,39.980,KWIATKOWSKI Michał,ALBASINI Michael,...,Sprint à deux,2017-04-16,p2,92,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,NaN
1,95,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,Astana Pro Team,6:35:07,39.940,KREUZIGER Roman,GASPAROTTO Enrico,...,Sprint à deux,2018-04-15,p2,103,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,NaN
2,96,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,Corendon - Circus,6:28:18,41.056,CLARKE Simon,FUGLSANG Jakob,...,Sprint of small group,2019-04-21,p2,108,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,NaN
3,97,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,Team Jumbo-Visma,5:03:29,43.218,PIDCOCK Thomas,SCHACHMANN Maximilian,...,Sprint of small group,2021-04-18,p2,81,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,NaN
4,98,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,INEOS Grenadiers,6:01:19,42.196,COSNEFROY Benoît,BENOOT Tiesj,...,Sprint à deux,2022-04-10,p2,112,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,2019stradebianche,Strade Bianche,2019,ALAPHILIPPE Julian,Deceuninck - Quick Step,4:47:14,38.436,FUGLSANG Jakob,VAN AERT Wout,...,Sprint à deux,2019-03-09,p3,151,Siena,Siena,NaN,1900-01-01 10:45:00,2019-03-09 10:45:00,NaN
96,0,2020stradebianche,Strade Bianche,2020,VAN AERT Wout,Team Jumbo-Visma,4:58:56,36.930,FORMOLO Davide,SCHACHMANN Maximilian,...,12.5 km solo,2020-08-01,p3,159,Siena,Siena,NaN,1900-01-01 13:45:00,2020-08-01 13:45:00,NaN
97,1,2021stradebianche,Strade Bianche,2021,VAN DER POEL Mathieu,Alpecin - Fenix,4:40:29,39.360,ALAPHILIPPE Julian,BERNAL Egan,...,0.5 km solo,2021-03-06,p3,159,Siena,Siena,NaN,1900-01-01 11:45:00,2021-03-06 11:45:00,NaN
98,2,2022stradebianche,Strade Bianche,2022,POGAČAR Tadej,UAE Team Emirates,4:47:49,38.358,VALVERDE Alejandro,ASGREEN Kasper,...,49.2 km solo,2022-03-05,p3,167,Siena,Siena,NaN,1900-01-01 11:45:00,2022-03-05 11:45:00,NaN


In [9]:
race_name_dict = baseline['Race Name'].to_dict()

In [11]:
race_name_dict

{0: 'Amstel Gold Race',
 1: 'Amstel Gold Race',
 2: 'Amstel Gold Race',
 3: 'Amstel Gold Race',
 4: 'Amstel Gold Race',
 5: 'Amstel Gold Race',
 6: 'Brussels Cycling Classic',
 7: 'Brussels Cycling Classic',
 8: 'Brussels Cycling Classic',
 9: 'Brussels Cycling Classic',
 10: 'Brussels Cycling Classic',
 11: 'Brussels Cycling Classic',
 12: 'Brussels Cycling Classic',
 13: 'De Brabantse Pijl',
 14: 'De Brabantse Pijl',
 15: 'De Brabantse Pijl',
 16: 'De Brabantse Pijl',
 17: 'De Brabantse Pijl',
 18: 'De Brabantse Pijl',
 19: 'De Brabantse Pijl',
 20: 'Dwars door Vlaanderen',
 21: 'Dwars door Vlaanderen',
 22: 'Dwars door Vlaanderen',
 23: 'Dwars door Vlaanderen',
 24: 'Dwars door Vlaanderen',
 25: 'Dwars door Vlaanderen',
 26: 'E3 Binckbank Classic',
 27: 'E3 Saxo Bank Classic',
 28: 'E3 Saxo Bank Classic',
 29: 'E3 Saxo Classic',
 30: 'Gent-Wevelgem in Flanders Fields',
 31: 'Gent-Wevelgem in Flanders Fields',
 32: 'Gent-Wevelgem in Flanders Fields',
 33: 'Gent-Wevelgem in Flanders F

In [33]:
race_name_mapping = dict(zip(baseline["PCS race entry"], baseline["Race Name"]))

In [13]:
import os
import gpxpy

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

def extract_gpx_coordinates(gpx_file, num_samples=10):
    with open(gpx_file, "r") as f:
        gpx = gpxpy.parse(f)
    
    coordinates = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coordinates.append((point.latitude, point.longitude))
                if len(coordinates) == num_samples:
                    return coordinates  
    return coordinates 

gpx_data = {}

for file in os.listdir(gpx_folder):
    if file.endswith(".gpx"):
        raw_name = file.replace(".gpx", "").strip().lower()  
        formatted_name = raw_name.replace("_", "").replace(" ", "") 
        gpx_path = os.path.join(gpx_folder, file)
        gpx_data[formatted_name] = extract_gpx_coordinates(gpx_path)

gpx_df = pd.DataFrame(list(gpx_data.items()), columns=["PCS race entry", "race_coords"])

baseline = baseline.merge(gpx_df, on='PCS race entry', how='left')

In [105]:
gpx_df

,PCS race entry
0,2017 amstel gold race
1,2017 brussels cycling classic
2,2017 de brabantse pijl - la fleche brabanconne
3,2017 dwars door vlaanderen
4,2017 gent-wevelgem in flanders fields
...,...
96,2023 omloop het nieuwsblad
97,2023 paris-roubaix
98,2023 ronde van vlaanderen - tour des flandres
99,2023 scheldeprijs


In [103]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,Team,Winner Time,Avg Speed (km/h),2nd Place,3rd Place,...,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race coords,race_coords,avg_wind_speed
0,94,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,Quick-Step Floors,6:31:40,39.980,KWIATKOWSKI Michał,ALBASINI Michael,...,p2,92,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,NaN,"[(50.87808, 5.71738), (50.87819, 5.71747), (50...",NaN
1,95,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,Astana Pro Team,6:35:07,39.940,KREUZIGER Roman,GASPAROTTO Enrico,...,p2,103,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,NaN,"[(50.87808, 5.71738), (50.87819, 5.71747), (50...",NaN
2,96,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,Corendon - Circus,6:28:18,41.056,CLARKE Simon,FUGLSANG Jakob,...,p2,108,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,NaN,"[(50.86287, 5.70027), (50.86307, 5.70027), (50...",NaN
3,97,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,Team Jumbo-Visma,5:03:29,43.218,PIDCOCK Thomas,SCHACHMANN Maximilian,...,p2,81,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,NaN,"[(50.86699, 5.78322), (50.86697, 5.78322), (50...",NaN
4,98,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,INEOS Grenadiers,6:01:19,42.196,COSNEFROY Benoît,BENOOT Tiesj,...,p2,112,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,NaN,"[(50.86307, 5.70026), (50.86312, 5.70026), (50...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,2019stradebianche,Strade Bianche,2019,ALAPHILIPPE Julian,Deceuninck - Quick Step,4:47:14,38.436,FUGLSANG Jakob,VAN AERT Wout,...,p3,151,Siena,Siena,NaN,1900-01-01 10:45:00,2019-03-09 10:45:00,NaN,"[(43.31194, 11.31732), (43.31184, 11.31728), (...",NaN
96,0,2020stradebianche,Strade Bianche,2020,VAN AERT Wout,Team Jumbo-Visma,4:58:56,36.930,FORMOLO Davide,SCHACHMANN Maximilian,...,p3,159,Siena,Siena,NaN,1900-01-01 13:45:00,2020-08-01 13:45:00,NaN,"[(43.31122, 11.3171), (43.31121, 11.3171), (43...",NaN
97,1,2021stradebianche,Strade Bianche,2021,VAN DER POEL Mathieu,Alpecin - Fenix,4:40:29,39.360,ALAPHILIPPE Julian,BERNAL Egan,...,p3,159,Siena,Siena,NaN,1900-01-01 11:45:00,2021-03-06 11:45:00,NaN,"[(43.31122, 11.3171), (43.31121, 11.3171), (43...",NaN
98,2,2022stradebianche,Strade Bianche,2022,POGAČAR Tadej,UAE Team Emirates,4:47:49,38.358,VALVERDE Alejandro,ASGREEN Kasper,...,p3,167,Siena,Siena,NaN,1900-01-01 11:45:00,2022-03-05 11:45:00,NaN,"[(43.31086, 11.31682), (43.3108, 11.31679), (4...",NaN


In [35]:
gpx_df["Race Name"] = gpx_df["PCS race entry"].map(race_name_mapping)

In [39]:
gpx_df['Year'] = gpx_df['PCS race entry'].str.extract(r'(\d{4})').astype(int)

In [45]:
gpx_df.loc[98, "Race Name"] = "Ronde van Vlaanderen"

In [63]:
gpx_df = gpx_df.sort_values(by=['Race Name', 'Year'], ascending=[True, True])

In [41]:
subset_baseline = baseline[['PCS race entry']]
subset_gpx = gpx_df[['PCS race entry']]

In [59]:
gpx_df[gpx_df.isnull().any(axis=1)]

,PCS race entry,race_coords,Race Name,Year


In [57]:
gpx_df.loc[49, "Race Name"] = "Liege - Bastogne - Liege"
gpx_df.loc[68, "Race Name"] = "Ronde van Vlaanderen"

In [71]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [113]:
import time
import os
from meteostat import Point, Hourly

In [115]:
def extract_points(gpx_file):
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)

    points = gpx.tracks[0].segments[0].points
    start = points[0]
    mid = points[len(points) // 2]
    end = points[-1]

    return [
        (start.latitude, start.longitude),
        (mid.latitude, start.longitude),
        (end.latitude, start.longitude)
    ]

gpx_folder = r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\data\euro classics gpx"

gpx_files = [os.path.join(gpx_folder, file) for file in os.listdir(gpx_folder) if file.endswith('.gpx')]
total_points = {file: extract_points(file) for file in gpx_files}

{'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Amstel Gold Race.gpx': [(50.87808,
   5.71738),
  (50.7649, 5.71738),
  (50.85835, 5.71738)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Brussels Cycling Classic.gpx': [(50.83958,
   4.39717),
  (50.60256, 4.39717),
  (50.89434, 4.39717)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 De Brabantse Pijl - La Fleche Brabanconne.gpx': [(50.87608,
   4.70039),
  (50.67499, 4.70039),
  (50.75711, 4.70039)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Dwars Door Vlaanderen .gpx': [(50.92555,
   3.11799),
  (50.84558, 3.11799),
  (50.87838, 3.11799)],
 'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Gent-Wevelgem in Flanders Fields

In [125]:
import requests

def get_wind_data(lat, lon):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=windspeed_10m,winddirection_10m&timezone=auto"
    
    response = requests.get(url)
    data = response.json()

    wind_speed = data["hourly"]["windspeed_10m"][0] 
    wind_direction = data["hourly"]["winddirection_10m"][0]

    return wind_speed, wind_direction

In [133]:
results = []

for file, points in total_points.items():
    for i, (lat, lon) in enumerate(points):
        wind_speed, wind_dir = get_wind_data(lat, lon)
        results.append([file, i, lat, lon, wind_speed, wind_dir])

windspeed_df = pd.DataFrame(results, columns=["File", "Point_Index", "Latitude", "Longitude", "Wind_Speed_kmh", "Wind_Direction"])
print(df)

df.to_csv("wind_speed.csv", index=False)

                                                  File  Point_Index  Latitude  \
0    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            0  50.87808   
1    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            1  50.76490   
2    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            2  50.85835   
3    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            0  50.83958   
4    C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            1  50.60256   
..                                                 ...          ...       ...   
298  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            1  51.40166   
299  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            2  51.25084   
300  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            0  43.31086   
301  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            1  43.16576   
302  C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...            2  43.31839   

     Longitude  Wind_Speed_

In [245]:
windspeed = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\wind_speed.xlsx")

In [247]:
windspeed['Compass direction'] = ''

In [249]:
def get_wind_direction(degrees):
    if 0 <= degrees < 11.25 or 348.75 <= degrees <= 360:
        return "N"
    elif 11.25 <= degrees < 33.75:
        return "NNE"
    elif 33.75 <= degrees < 56.25:
        return "NE"
    elif 56.25 <= degrees < 78.75:
        return "ENE"
    elif 78.75 <= degrees < 101.25:
        return "E"
    elif 101.25 <= degrees < 123.75:
        return "ESE"
    elif 123.75 <= degrees < 146.25:
        return "SE"
    elif 146.25 <= degrees < 168.75:
        return "SSE"
    elif 168.75 <= degrees < 191.25:
        return "S"
    elif 191.25 <= degrees < 213.75:
        return "SSW"
    elif 213.75 <= degrees < 236.25:
        return "SW"
    elif 236.25 <= degrees < 258.75:
        return "WSW"
    elif 258.75 <= degrees < 281.25:
        return "W"
    elif 281.25 <= degrees < 303.75:
        return "WNW"
    elif 303.75 <= degrees < 326.25:
        return "NW"
    elif 326.25 <= degrees < 348.75:
        return "NNW"
    else:
        return "Invalid"

def convert_wind_direction_df(df, wind_col='Wind_Direction'):
        if wind_col in df.columns:
            df['Compass direction'] = df[wind_col].apply(get_wind_direction)

convert_wind_direction_df(windspeed)

In [251]:
windspeed.reset_index()

,index,File,Point_Index,Latitude,Longitude,Wind_Speed_kmh,Wind_Direction,Compass direction
0,0,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,0,50.87808,5.71738,12.2,80,E
1,1,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,1,50.76490,5.71738,9.0,71,ENE
2,2,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,2,50.85835,5.71738,12.2,78,ENE
3,3,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,0,50.83958,4.39717,14.4,92,E
4,4,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,1,50.60256,4.39717,15.5,87,E
...,...,...,...,...,...,...,...,...
298,298,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,1,51.40166,3.80118,22.7,103,ESE
299,299,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,2,51.25084,3.80118,14.0,104,ESE
300,300,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,0,43.31086,11.31682,4.0,100,E
301,301,C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Pro...,1,43.16576,11.31682,6.1,62,ENE


In [241]:
df.at[0, 'File']

'C:\\Users\\mrmik\\OneDrive\\Desktop\\NSS\\Python\\Projects\\Cycling_Capstone\\data\\euro classics gpx\\2017 Amstel Gold Race.gpx'

In [253]:
windspeed['File'] = windspeed['File'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

In [257]:
windspeed.to_excel('windspeed.xlsx')